In [1]:
import os 
import re
import time
import pandas as pd
import multiprocessing as mp
import numpy as np
import dask.dataframe as dd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import OrderedDict
import torch.nn.functional as F
from torch import nn
import math
import matplotlib.pyplot as plt

from statistics import mean

import torch
from torch import optim


import os

path0 ='../'
df = pd.read_csv(path0+"Dataset_S022Final.csv")

a=time.time()
df=df[['Next_Current_diff','Next_Pre_diff','SNext_Current_diff','SNext_Pre_diff','rcvdPK','duration(ms)','packet_type','droppedPKWrongPort','sentPK','size','channel','DataQueueLen','passedUpPk','rcvdPKFromHL','rcvdPKFromLL','sentDownPK','DropPKByQueue','snir','throughput','label']]         


df_Normal=df[0:76064].copy()
#print(df_Normal)
df_UDP=df[76064:214037].copy()
#print(df_UDP)
df_pluies=df[214037:426866].copy()

#print(df_Normal2)
df_jam=df[462481:-1].copy()

X=df_Normal.drop(columns = ['label']).copy()
y=df_Normal[['label']].copy()
#print(y.columns)
X1=df_UDP.drop(columns = ['label']).copy()
y1=df_UDP[['label']].copy()
X2=df_pluies.drop(columns = ['label']).copy()
y2=df_pluies[['label']].copy()
X3=df_jam.drop(columns = ['label']).copy()
y3=df_jam[['label']].copy()

train=0.4


xcol=X.columns
ycol=y.columns


X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=train,shuffle=False)
X_train1, X_rem1, y_train1, y_rem1 = train_test_split(X1,y1, train_size=train,shuffle=False)
X_train2, X_rem2, y_train2, y_rem2 = train_test_split(X2,y2, train_size=train,shuffle=False)
X_train3, X_rem3, y_train3, y_rem3 = train_test_split(X3,y3, train_size=train,shuffle=False)

validation=0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5,shuffle=False)
X_valid1, X_test1, y_valid1, y_test1 = train_test_split(X_rem1,y_rem1, test_size=0.5,shuffle=False)
X_valid2, X_test2, y_valid2, y_test2 = train_test_split(X_rem2,y_rem2, test_size=0.5,shuffle=False)
X_valid3, X_test3, y_valid3, y_test3 = train_test_split(X_rem3,y_rem3, test_size=0.5,shuffle=False)


X_train=np.concatenate((X_train, X_train1, X_train2,X_train3))
X_valid=np.concatenate((X_valid, X_valid1, X_valid2,X_valid3))
X_test=np.concatenate((X_test, X_test1, X_test2,X_test3))

y_train=np.concatenate((y_train, y_train1, y_train2,y_train3))
y_valid=np.concatenate((y_valid, y_valid1, y_valid2,y_valid3))
y_test=np.concatenate((y_test, y_test1, y_test2,y_test3))

X_train = pd.DataFrame(X_train, columns= xcol)
X_valid = pd.DataFrame(X_valid, columns= xcol)
X_test = pd.DataFrame(X_test, columns= xcol)

y_train = pd.DataFrame(y_train, columns= ycol)
y_valid = pd.DataFrame(y_valid, columns= ycol)
y_test = pd.DataFrame(y_test, columns= ycol)




a-time.time()
X_train1=X_train.values
y_train1=y_train.values
#X_train1=X_train1.astype(float)

torch_tensor = torch.tensor(X_train1)
#torch_tensor[torch_tensor != torch_tensor]=float(0)
t=[]
test=0
test0=0

for j in y_train1:
    i=j[0]
#    print(i)
    if (i=='Normal'):
        t.append(int(0))
        test0+=1
    #t.append(int(1))
    if (i=='DDOS_UDP_FLOOD'):
        t.append(int(1))
    if (i=='PLUIES_ET_ORAGES'):
        t.append(int(2))
        test+=1
    if (i=='BROUILLAGE_Trafic'):
        t.append(int(3))

labels = torch.LongTensor(t)

bigX = X_valid
#[['Next_Current_diff','Next_Pre_diff','SNext_Current_diff','SNext_Pre_diff','rcvdPK','duration(ms)','droppedPKWrongPort','sentPK','size','throughput']]  ,'channel'       
bigY = y_valid['label']
#print(bigY)
X_valid1=bigX.values
y_valid1=bigY.values

vtorch_tensor = torch.tensor(X_valid1)
v=[]

for i in y_valid1:
    #i=j[0]
    #print(i)
    if (i=='Normal'):
        v.append(int(0))
        #t.append(int(1))
    if (i=='DDOS_UDP_FLOOD'):
        v.append(int(1))
    if (i=='PLUIES_ET_ORAGES'):
        v.append(int(2))
    if (i=='BROUILLAGE_Trafic'):
        v.append(int(3))

vlabels = torch.LongTensor(v)


X_test1=X_test.values
y_test1=y_test.values

#X_train1=X_train1.astype(float)

ttorch_tensor = torch.tensor(X_test1)
t=[]

for j in y_test1:
    i=j[0]
#    print(i)
    if (i=='Normal'):
        t.append(int(0))
        #t.append(int(1))
    if (i=='DDOS_UDP_FLOOD'):
        t.append(int(1))
    if (i=='PLUIES_ET_ORAGES'):
        t.append(int(2))
    if (i=='BROUILLAGE_Trafic'):
        t.append(int(3))

tlabels = torch.LongTensor(t)


 

In [2]:
torch.set_default_dtype(torch.float64)

from torch import nn, optim
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
# input_size = 784 # 28x28
num_classes = 4
#num_epochs = 2
batch_size = 100
learning_rate = 0.02

input_size = 19
sequence_length = 28
hidden_size = 132
num_layers = 2

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers,batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        
        # or:
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        x=x.view(-1,1,19)
        #print(x.shape)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        
        #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        out, _ = self.rnn(x, (h0,c0) )  
        # or:
        #out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
         
        #out = self.fc(out)
        x = F.log_softmax(self.fc(out), dim=1)
        # out: (n, 10)
        return x

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)


class My_module(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,8,3,padding=1)
        self.conv2=nn.Conv2d(8,12,3,padding=1)
        self.conv3=nn.Conv2d(12,18,3,padding=1)
        self.fc1 = nn.Linear(18*6*3,350)
        self.fc2 = nn.Linear(350,400)
        self.fc3 = nn.Linear(56, 64)
        self.fc4 = nn.Linear(64, 4)
        self.fc5 = nn.Linear(400, 4)
        self.dropout = nn.Dropout(p=0.0)
        self.pool=nn.MaxPool2d(1,1)
        
    def forward(self, x):
        # make sure input tensor is flattened
        #x = x.view(x.shape[0], -1)
        #print(x.shape)
        
        x=self.pool(F.relu(self.conv1(x)))
        #print(x.shape)

        x=self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        
        x=self.pool(F.relu(self.conv3(x)))
        #print(x.shape)
        
        x=x.view(-1,18*6*3)
        #print(x.shape)
        x = self.dropout(F.relu(self.fc1(x)))
        #print(x.shape)
        
        x = self.dropout(F.relu(self.fc2(x)))
        #x = self.dropout(F.relu(self.fc3(x))) 
        #x = self.dropout(F.relu(self.fc4(x))) 
    
        x = F.log_softmax(self.fc5(x), dim=1)        #print(x.shape)        
        return x
Fonction_de_perte = nn.NLLLoss()

optimizer = optim.SGD(model.parameters(), lr=0.003)

model.load_state_dict(torch.load('models\\dataset1_1_50_LSTM.pth'))

#indices = [0,3, 299:303]


<All keys matched successfully>

In [4]:

a=time.time()
summm=[]
sum1=[]
sum2=[]
sum3=[]
sum4=[]

tinputs =ttorch_tensor[:,:] 
tlabels1=tlabels[:]


#tinputs =torch_tensor[:,:] 
#tlabels1=labels[:]

print('=====================')
b=time.time()
output = torch.exp(model(tinputs))
test_loss=Fonction_de_perte(output, tlabels1)
top_p , top_c = output.topk(1, dim=1)
propabilities= torch.mean(top_p.type(torch.FloatTensor))*100
equals = top_c==tlabels1.view(*top_c.shape)
accuracy = torch.mean(equals.type(torch.FloatTensor))
model.train()
print ('test accuracy :{0:.8f} with total prob : {3:.8f} and  test loss : {2:.8f} ,  time {1:.8f} '.format(accuracy*100 , time.time()-b ,test_loss ,propabilities))

class_correct = list(0. for i in range (4))
class_total = list(0. for i in range (4))

C_n_udp=0
C_n_pluies=0
C_n_jam=0
C_n_total=0


C_u_normal=0
C_u_jam=0
C_u_pluies=0
C_u_total=0

C_p_normal=0
C_p_udp=0
C_p_jam=0
C_p_total=0

C_j_normal=0
C_j_udp=0
C_j_pluies=0
C_j_total=0

for i, x in enumerate(tinputs):
    optimizer.zero_grad()
    x2=x[None,:]
    with torch.no_grad():
        output = torch.exp(model(x2))
    out=output.detach().numpy()*100
 
    l3=tlabels1[i].item()
  
    if(l3==0):
        summm.append(out[0][0])
        sum1.append(out[0][0])
        if(top_c[i][0]!=l3):
            if(top_c[i][0]==1):
                C_n_udp+=1
            if(top_c[i][0]==2):
                C_n_pluies+=1
            if(top_c[i][0]==3):
                C_n_jam +=1
            
            C_n_total +=1
            
    if(l3==1):
        summm.append(out[0][1])
        sum2.append(out[0][1])
        if(top_c[i][0]!=l3):
            if(top_c[i][0]==3):
                C_u_jam+=1
            if(top_c[i][0]==2):
                C_u_pluies+=1
            if(top_c[i][0]==0):
                C_u_normal +=1
            
            C_u_total +=1

    if(l3==2):
        summm.append(out[0][2])
        sum3.append(out[0][2])
        if(top_c[i][0]!=l3):
            if(top_c[i][0]==1):
                C_p_udp+=1
            if(top_c[i][0]==3):
                C_p_jam+=1
            if(top_c[i][0]==0):
                C_p_normal +=1
            
            C_p_total +=1

    if(l3==3):
        summm.append(out[0][3])
        sum4.append(out[0][3])
        if(top_c[i][0]!=l3):
            if(top_c[i][0]==1):
                C_j_udp+=1
            if(top_c[i][0]==2):
                C_j_pluies+=1
            if(top_c[i][0]==0):
                C_j_normal +=1
            
            C_j_total +=1

    class_total[l3]+=1
    class_correct[l3]+=equals[i][0]
    #print(output)
print('la précision de detection globale: {0} '.format(mean(summm)))
print('detection des communication normal: {0:.8f} with accuracy {1:.8f} ( {2:.0f}/{3:.0f} )'.format(mean(sum1), class_correct[0]*100/class_total[0] ,class_correct[0] ,class_total[0]))
if(C_n_total!=0):
    print('details normal classed udp :{0:.5f}, pluies {1:.5f} , jam {2:.5f} (ou {3:.5f},{4:.5f},{5:.5f}) '.format(C_n_udp*100/class_total[0],C_n_pluies*100/class_total[0],C_n_jam*100/class_total[0],C_n_udp*100/C_n_total,C_n_pluies*100/C_n_total,C_n_jam*100/C_n_total))
else:
    print('detection normal 100')

print('detection du deni de service par udp flood {0:.8f} with accuracy {1:.8f} ( {2:.0f}/{3:.0f} )'.format(mean(sum2), class_correct[1]*100/class_total[1] ,class_correct[1] ,class_total[1]))
if(C_u_total!=0):
    print('details udp classed normal :{0:.5f}, pluies {1:.5f} , jam {2:.5f} (ou {3:.5f},{4:.5f},{5:.5f}) '.format(C_u_normal*100/class_total[1],C_u_pluies*100/class_total[1],C_u_jam*100/class_total[1],C_u_normal*100/C_u_total,C_u_pluies*100/C_u_total,C_u_jam*100/C_u_total))
else:
    print('detection udp flood 100')
    
print('detection du deni naturel : pluies et orages : {0:.8f} with accuracy {1:.8f} ( {2:.0f}/{3:.0f} )'.format(mean(sum3), class_correct[2]*100/class_total[2] ,class_correct[2] ,class_total[2]))
if(C_p_total!=0):
    print('details pluies classed udp :{0:.5f}, normal {1:.5f} , jam {2:.5f} (ou {3:.5f},{4:.5f},{5:.5f}) '.format(C_p_udp*100/class_total[2],C_p_normal*100/class_total[2],C_p_jam*100/class_total[2],C_p_udp*100/C_p_total,C_p_normal*100/C_p_total,C_p_jam*100/C_p_total))
else:
    print('detection pluies et orages 100')

print('detection du deni naturel : jam : {0:.8f} with accuracy {1:.8f} ( {2:.0f}/{3:.0f} )'.format(mean(sum4), class_correct[3]*100/class_total[3] ,class_correct[3] ,class_total[3]))
if(C_j_total!=0):
    print('details jam classed udp :{0:.5f}, pluies {1:.5f} , normal {2:.5f} (ou {3:.5f},{4:.5f},{5:.5f}) '.format(C_j_udp*100/class_total[3],C_j_pluies*100/class_total[3],C_j_normal*100/class_total[3],C_j_udp*100/C_j_total,C_j_pluies*100/C_j_total,C_j_normal*100/C_j_total))
else:
    print('detection brouillage 100')
print('=====================')

print('<================================>')
print('Total_time')
print(time.time()-a)

test accuracy :98.71617889 with total prob : 99.65184021 and  test loss : -0.98714451 ,  time 11.60582447 
la précision de detection globale: 98.7144510256901 
detection des communication normal: 97.80467222 with accuracy 97.73304114 ( 78938/80769 )
details normal classed udp :0.00000, pluies 2.06515 , jam 0.20181 (ou 0.00000,91.09776,8.90224) 
detection du deni de service par udp flood 99.99386580 with accuracy 99.99161426 ( 35772/35775 )
details udp classed normal :0.00839, pluies 0.00000 , jam 0.00000 (ou 100.00000,0.00000,0.00000) 
detection du deni naturel : pluies et orages : 99.71147213 with accuracy 99.97806344 ( 22788/22793 )
details pluies classed udp :0.00000, normal 0.02194 , jam 0.00000 (ou 0.00000,100.00000,0.00000) 
detection du deni naturel : jam : 99.99056030 with accuracy 100.00000000 ( 3907/3907 )
detection brouillage 100
<================================>
Total_time
94.15830945968628


In [1]:
import os 
import re
import time
import pandas as pd
import multiprocessing as mp
import numpy as np
import dask.dataframe as dd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import OrderedDict
import torch.nn.functional as F
from torch import nn
import math
import matplotlib.pyplot as plt

from statistics import mean

import torch
from torch import optim


from torch import nn, optim
import torch.nn.functional as F
import os

def Test():
    torch.set_default_dtype(torch.float64)



    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Hyper-parameters 
    # input_size = 784 # 28x28
    num_classes = 4
    #num_epochs = 2
    batch_size = 100
    learning_rate = 0.02

    input_size = 19
    sequence_length = 28
    hidden_size = 132
    num_layers = 2

    class RNN(nn.Module):
        def __init__(self, input_size, hidden_size, num_layers, num_classes):
            super(RNN, self).__init__()
            self.num_layers = num_layers
            self.hidden_size = hidden_size
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers,batch_first=True)
            # -> x needs to be: (batch_size, seq, input_size)

            # or:
            #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
            #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
            self.fc = nn.Linear(hidden_size, num_classes)

        def forward(self, x):
            # Set initial hidden states (and cell states for LSTM)
            x=x.view(-1,1,19)
            #print(x.shape)
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 


            #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 

            # x: (n, 28, 28), h0: (2, n, 128)

            # Forward propagate RNN
            out, _ = self.rnn(x, (h0,c0) )  
            # or:
            #out, _ = self.lstm(x, (h0,c0))  

            # out: tensor of shape (batch_size, seq_length, hidden_size)
            # out: (n, 28, 128)

            # Decode the hidden state of the last time step
            out = out[:, -1, :]
            # out: (n, 128)

            #out = self.fc(out)
            x = F.log_softmax(self.fc(out), dim=1)
            # out: (n, 10)
            return x

    model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)


    class My_module(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1=nn.Conv2d(1,8,3,padding=1)
            self.conv2=nn.Conv2d(8,12,3,padding=1)
            self.conv3=nn.Conv2d(12,18,3,padding=1)
            self.fc1 = nn.Linear(18*6*3,350)
            self.fc2 = nn.Linear(350,400)
            self.fc3 = nn.Linear(56, 64)
            self.fc4 = nn.Linear(64, 4)
            self.fc5 = nn.Linear(400, 4)
            self.dropout = nn.Dropout(p=0.0)
            self.pool=nn.MaxPool2d(1,1)

        def forward(self, x):
            # make sure input tensor is flattened
            #x = x.view(x.shape[0], -1)
            #print(x.shape)

            x=self.pool(F.relu(self.conv1(x)))
            #print(x.shape)

            x=self.pool(F.relu(self.conv2(x)))
            #print(x.shape)

            x=self.pool(F.relu(self.conv3(x)))
            #print(x.shape)

            x=x.view(-1,18*6*3)
            #print(x.shape)
            x = self.dropout(F.relu(self.fc1(x)))
            #print(x.shape)

            x = self.dropout(F.relu(self.fc2(x)))
            #x = self.dropout(F.relu(self.fc3(x))) 
            #x = self.dropout(F.relu(self.fc4(x))) 

            x = F.log_softmax(self.fc5(x), dim=1)        #print(x.shape)        
            return x
    Fonction_de_perte = nn.NLLLoss()

    optimizer = optim.SGD(model.parameters(), lr=0.003)

    model.load_state_dict(torch.load('models\\dataset1_1_50_LSTM.pth'))

    #indices = [0,3, 299:303]



    path0 ='D:\\share\\Data_file\\Senarios\\new_test\\satellite00\\Senario_test_data_normalized.csv' 
    df = pd.read_csv(path0)
    df1=df[['Next_Current_diff','Next_Pre_diff','SNext_Current_diff','SNext_Pre_diff','rcvdPK','duration(ms)','packet_type','droppedPKWrongPort','sentPK','size','channel','DataQueueLen','passedUpPk','rcvdPKFromHL','rcvdPKFromLL','sentDownPK','DropPKByQueue','snir','throughput']]         

    #df1=df[['channel','Next_Current_diff','Next_Pre_diff','size','packet_type','SNext_Current_diff','SNext_Pre_diff','snir','Flow Bytes_s','Flow Packets_s','meanT_b_2P','minT_b_2P','maxT_b_2P']]     
    np_data=df.to_numpy()
    df1=df1.to_numpy()
    ttorch_tensor = torch.tensor(df1)
    tinputs =ttorch_tensor[:,:] 
    output = torch.exp(model(tinputs))
    #test_loss=Fonction_de_perte(output, tlabels1)
    top_p , top_c = output.topk(1, dim=1)
    node=set()
    node1=set()
    node2=dict()
    node10=dict()
    node11=set()


    #with open(location+"results/index.txt", "w") as f:
     #   #f.write(str(C_index+1))
      #  f.close()
    name='Pluies'
    if(name=='Pluies'):
        print('pluies')
        print(len(top_c))
        top_c = top_c.cpu().detach().numpy()
        for index,i in enumerate(top_c):
            i=i[0]
            i1=np_data[index][15]+' '+np_data[index][17]

            if (i1 in node11):
                    pass
                    node10[i1]+=1
                    #print(node2[i]) 
            else:
                    node11.add(i1)
                    #[sentTime , n]
                    node10[i1]=1
            if(i==2):
                #print('udp attack from '+str(np_data[index][5])+' and with probability of '+str(top_p[index]))
                i1=np_data[index][15]+' '+np_data[index][17]
                if (i1 in node):
                    pass
                    node2[i1][1]+=1
                    #print(node2[i]) 
                else:
                    node.add(i1)
                    #[sentTime , n]
                    node2[i1]=list([np_data[index][1],1])

        somme=0
        max1=0
        for i in node2:
            node2[i].append(node2[i][1]*100/len(top_c))
            somme+=node2[i][1]*100/len(top_c)
            if(max1<(node2[i][1]*100/len(top_c))):
                max1=node2[i][1]*100/len(top_c)
        if(somme>40 or 1):

            print('=========================================================================================================')
            print('                                   Pluies et Orages (Brouillage Naturel)')
            print('=========================================================================================================')
            #now = datetime.now()
            #print(' Detection entre :'+str(dt_0)+' et ' + str(now.strftime(" %H:%M:%S")))
            for i in node2:
                print('Flux entre '+str(i.replace(' ', ' et '))+' avec une probabilite de '+str("{:.2f}".format(node2[i][1]*100/node10[i])) + '% ( '+ str("{:.2f}".format(node2[i][2])) +' % de flux capture )'  )
    node=set()
    node1=set()
    node2=dict()
    node10=dict()
    node11=set()

    name='Jam'
    if(name=='Jam'):
        #print(location)
        #print(np_data[0])
        for index,i in enumerate(top_c):
            i=i[0]
            i1=np_data[index][15]+' '+np_data[index][17]
            if (i1 in node11):
                    pass
                    node10[i1]+=1
                    #print(node2[i]) 
            else:
                    node11.add(i1)
                    #[sentTime , n]
                    node10[i1]=1
            if(i==3):

                #print('udp attack from '+str(np_data[index][5])+' and with probability of '+str(top_p[index]))
                i1=np_data[index][15]+' '+np_data[index][17]
                if (i1 in node):
                    pass
                    node2[i1][1]+=1
                    #print(node2[i]) 
                else:
                    node.add(i1)
                    #[sentTime , n]
                    node2[i1]=list([np_data[index][1],1])
        somme=0
        max1=0
        for i in node2:
            node2[i].append(node2[i][1]*100/len(top_c))
            somme+=node2[i][1]*100/len(top_c)
            if(max1<(node2[i][1]*100/len(top_c))):
                max1=node2[i][1]*100/len(top_c)
        if(somme>35 or 1):

            print('=========================================================================================================')
            print('                                              Jamming')
            print('=========================================================================================================')
           # now = datetime.now()
            #print(' Detection entre :'+str(dt_0)+' et ' + str(now.strftime(" %H:%M:%S")))

            for i in node2:
                print('Flux entre '+str(i.replace(' ', ' et '))+' avec une probabilite de '+str("{:.2f}".format(node2[i][1]*100/node10[i])) + ' % ( '+ str("{:.2f}".format(node2[i][2])) +' % de flux capture )'  )
    node=set()
    node1=set()
    node2=dict()
    node10=dict()
    node11=set()

    name='Flood'
    if(name=='Flood'):
        #print('flood')
        #top_c = top_c.cpu().detach().numpy()
        for index,i in enumerate(top_c):
            i=i[0]
            i1=np_data[index][15]+' '+np_data[index][17]

            if (i1 in node11):
                    pass
                    node10[i1]+=1
                    #print(node2[i]) 
            else:
                    node11.add(i1)
                    #[sentTime , n]
                    node10[i1]=1
            if(i==1):
                #print('udp attack from '+str(np_data[index][5])+' and with probability of '+str(top_p[index]))
                i1=np_data[index][15]+' '+np_data[index][17]
                if (i1 in node):
                    pass
                    node2[i1][1]+=1
                    #print(node2[i]) 
                else:
                    node.add(i1)
                    #[sentTime , n]
                    node2[i1]=list([np_data[index][1],1])

        somme=0
        max1=0
        for i in node2:
            node2[i].append(node2[i][1]*100/len(top_c))
            somme+=node2[i][1]*100/len(top_c)
            if(max1<(node2[i][1]*100/len(top_c))):
                max1=node2[i][1]*100/len(top_c)
        del(top_c)
        if(somme>20 or 1):

            print('=========================================================================================================')
            print('                                      Attaque par Flood')
            print('=========================================================================================================')

            #now = datetime.now()
            #print(' Detection entre :'+str(dt_0)+' et ' + str(now.strftime(" %H:%M:%S")))

            for i in node2:
                print('Flux entre '+str(i.replace(' ', ' et '))+' avec une probabilite de '+str("{:.2f}".format(node2[i][1]*100/node10[i])) + '% ( '+ str("{:.2f}".format(node2[i][2])) +' % de flux capture )'  )


In [ ]:
################################3

In [ ]:
################################3

In [ ]:
################################3

In [ ]:
################################3

In [ ]:
################################3

In [4]:
Test()

pluies
7379
                                   Pluies et Orages (Brouillage Naturel)
Flux entre 19:11:16:100 et 19:10:1:101 avec une probabilite de 2.42% ( 0.11 % de flux capture )
Flux entre 19:11:20:100 et 19:10:1:103 avec une probabilite de 1.11% ( 0.08 % de flux capture )
                                              Jamming
Flux entre 19:10:1:102 et 19:10:1:105 avec une probabilite de 57.97 % ( 1.63 % de flux capture )
Flux entre 19:10:1:104 et 19:10:1:106 avec une probabilite de 58.60 % ( 1.71 % de flux capture )
Flux entre 19:11:20:100 et 19:10:1:103 avec une probabilite de 11.62 % ( 0.85 % de flux capture )
Flux entre 19:11:16:100 et 19:10:1:101 avec une probabilite de 8.16 % ( 0.37 % de flux capture )
Flux entre 19:10:1:105 et 19:10:1:106 avec une probabilite de 63.64 % ( 1.42 % de flux capture )
Flux entre 19:11:12:100 et 19:10:1:107 avec une probabilite de 13.24 % ( 0.12 % de flux capture )
Flux entre 19:11:11:100 et 19:10:1:107 avec une probabilite de 14.75 % ( 0.12 % de fl